In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리

In [113]:
import pandas as pd
import numpy as np
import os

## path 설정

In [114]:
# path = '../..'
path = '/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더'
camb_word_path = f'{path}/words/raw_word_data/new'
origin_word_path = f'{path}/words/final_datasets'

## 결과 데이터프레임 

In [115]:
print(f'{origin_word_path}/words_levels_df.csv')
if os.path.exists(f'{origin_word_path}/words_levels_df.csv'):
    df_result = pd.read_csv(f'{origin_word_path}/words_levels_df.csv', index_col='Word')
    display(df_result)
else:
  print('not exists file');


/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/final_datasets/words_levels_df.csv


,oxford_level,oxford_version,lexile_grade,lexile_category,awsl,toefl,tsl,bsl,ngsl_freq,ngsl_sp_freq,naver_priority,lemmetized,proper_noun,word_level
Word,,,,,,,,,,,,,,
a,NN,0.0,0.0,NN,Y,N,N,N,A1,A1,2,True,False,2.0
aback,NN,0.0,0.0,NN,N,N,N,N,B3,C1,0,False,False,4.0
abacus,NN,0.0,0.0,NN,N,N,N,N,C3,C3,1,True,False,4.0
abalone,NN,0.0,0.0,NN,N,N,N,N,C3,NN,0,True,False,6.0
abandon,B2,3000.0,0.0,NN,N,N,N,N,A2,A3,2,True,False,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zygomorphic,NN,0.0,0.0,NN,N,N,N,N,C4,NN,0,False,False,6.0
zygosity,NN,0.0,0.0,NN,N,N,N,N,C4,NN,0,False,False,6.0
zygote,NN,0.0,6.0,science,N,N,N,N,C1,C3,0,True,False,5.0


## 단어 파일 불러와서 합치기

In [116]:
levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
df_camb = pd.DataFrame()
count = 0
for level in levels:
  file_name = f'{camb_word_path}/cam_{level}.csv' 
  print(f'{camb_word_path}/cam_{level}.csv')
  if os.path.exists(file_name):
    df = pd.read_csv(file_name, sep='\t', header=None)
    df.rename(columns={0:'Word', 2: 'camb_level'}, inplace=True)
    df.set_index('Word', inplace=True)
    df.drop(columns=[1, 3, 4, 5], axis=1, inplace=True)
    df_camb = pd.concat([df_camb, df])
    count += len(df)
  else:
    print('not exists file');

display(count, df_camb)


/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/raw_word_data/new/cam_A1.csv
/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/raw_word_data/new/cam_A2.csv
/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/raw_word_data/new/cam_B1.csv
/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/raw_word_data/new/cam_B2.csv
/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/raw_word_data/new/cam_C1.csv
/content/drive/MyDrive/elice_데이터분석_프로젝트/팀_공유_폴더/words/raw_word_data/new/cam_C2.csv


15385

,camb_level
Word,
clothes,A1
twelve,A1
two,A1
to,A1
twenty,A1
...,...
with bated breath,C2
be lost for words,C2
by word of mouth,C2


## 소문자로 변형

In [117]:
df_camb.index = df_camb.index.str.lower()
df_camb

,camb_level
Word,
clothes,A1
twelve,A1
two,A1
to,A1
twenty,A1
...,...
with bated breath,C2
be lost for words,C2
by word of mouth,C2


## sth, sb 제거

In [118]:
processed_df_camb = df_camb[~df_camb.index.str.contains('sth')]
processed_df_camb = processed_df_camb[~processed_df_camb.index.str.contains('sb')]
processed_df_camb

,camb_level
Word,
clothes,A1
twelve,A1
two,A1
to,A1
twenty,A1
...,...
with bated breath,C2
be lost for words,C2
by word of mouth,C2


## 특수문자(공백 제외) 제거

In [119]:
processed_df_camb = processed_df_camb[~processed_df_camb.index.str.contains(r'[^a-zA-Z0-9\ ]', regex=True)]
processed_df_camb


,camb_level
Word,
clothes,A1
twelve,A1
two,A1
to,A1
twenty,A1
...,...
with your bare hands,C2
with bated breath,C2
be lost for words,C2


## df_camb에서 중복 데이터 삭제

In [171]:
drop_duplicates_df_camb = processed_df_camb.reset_index()
drop_duplicates_df_camb = drop_duplicates_df_camb.drop_duplicates('Word', keep='last')
drop_duplicates_df_camb.set_index('Word', drop=True, inplace=True)
drop_duplicates_df_camb

,camb_level
Word,
clothes,A1
twelve,A1
two,A1
twenty,A1
three,A1
...,...
with your bare hands,C2
with bated breath,C2
be lost for words,C2


## df_result에 df_camb 추가

In [172]:
df_temp = df_result.join(drop_duplicates_df_camb, how='outer')
df_temp

,oxford_level,oxford_version,lexile_grade,lexile_category,awsl,toefl,tsl,bsl,ngsl_freq,ngsl_sp_freq,naver_priority,lemmetized,proper_noun,word_level,camb_level
Word,,,,,,,,,,,,,,,
a,NN,0.0,0.0,NN,Y,N,N,N,A1,A1,2.0,True,False,2.0,A2
a bit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B2
a bone of contention,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C2
a breath of fresh air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C2
a broken heart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zygomorphic,NN,0.0,0.0,NN,N,N,N,N,C4,NN,0.0,False,False,6.0,NaN
zygosity,NN,0.0,0.0,NN,N,N,N,N,C4,NN,0.0,False,False,6.0,NaN
zygote,NN,0.0,6.0,science,N,N,N,N,C1,C3,0.0,True,False,5.0,NaN
